# Unsupervised Learning Trading Strategy

* Download/Load SP500 stocks prices data.
* Calculate different features and indicators on each stock.
* Aggregate on monthly level and filter top 150 most liquid stocks.
* Calculate Monthly Returns for different time-horizons.
* Download Fama-French Factors and Calculate Rolling Factor Betas.
* For each month fit a K-Means Clustering Algorithm to group similar assets based on their features.
* For each month select assets based on the cluster and form a portfolio based on Efficient Frontier max sharpe ratio optimization.
* Visualize Portfolio returns and compare to SP500 returns.

## 1. Download/Load SP500 stocks prices data.

In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-09-16'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%***********************]  503 of 503 completed


Price                              adj close       close        high  \
date                      ticker                                       
2016-09-19 00:00:00+00:00 A        42.213081   44.950001   45.480000   
                          AAPL     26.191391   28.395000   29.045000   
                          ABBV     44.565331   62.910000   63.709999   
                          ABT      36.004757   41.680000   42.360001   
                          ACGL     26.823334   26.823334   26.930000   
...                                      ...         ...         ...   
2024-09-13 00:00:00+00:00 XYL     130.830002  130.830002  132.309998   
                          YUM     133.649994  133.649994  133.949997   
                          ZBH     106.260002  106.260002  107.940002   
                          ZBRA    337.480011  337.480011  343.510010   
                          ZTS     191.089996  191.089996  192.619995   

Price                                    low        open       volume  
date                      ticker                                       
2016-09-19 00:00:00+00:00 A        44.860001   44.900002    1927900.0  
                          AAPL     28.312500   28.797501  188092000.0  
                          ABBV     62.830002   63.619999    7903500.0  
                          ABT      41.590000   42.040001    7552700.0  
                          ACGL     26.553333   26.680000     662100.0  
...                                      ...         ...          ...  
2024-09-13 00:00:00+00:00 XYL     130.639999  131.690002     995300.0  
                          YUM     132.929993  133.479996    1811500.0  
                          ZBH     105.199997  105.199997    1460500.0  
                          ZBRA    337.190002  339.119995     267700.0  
                          ZTS     190.139999  190.570007    1725200.0  

[989567 rows x 6 columns]

## 2. Calculate features and technical indicators for each stock.

* Garman-Klass Volatility
* RSI
* Bollinger Bands
* ATR
* MACD
* Dollar Volume

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [3]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price                              adj close       close        high  \
date                      ticker                                       
2016-09-19 00:00:00+00:00 A        42.213081   44.950001   45.480000   
                          AAPL     26.191391   28.395000   29.045000   
                          ABBV     44.565331   62.910000   63.709999   
                          ABT      36.004757   41.680000   42.360001   
                          ACGL     26.823334   26.823334   26.930000   
...                                      ...         ...         ...   
2024-09-13 00:00:00+00:00 XYL     130.830002  130.830002  132.309998   
                          YUM     133.649994  133.649994  133.949997   
                          ZBH     106.260002  106.260002  107.940002   
                          ZBRA    337.480011  337.480011  343.510010   
                          ZTS     191.089996  191.089996  192.619995   

Price                                    low        open       volume  \
date                      ticker                                        
2016-09-19 00:00:00+00:00 A        44.860001   44.900002    1927900.0   
                          AAPL     28.312500   28.797501  188092000.0   
                          ABBV     62.830002   63.619999    7903500.0   
                          ABT      41.590000   42.040001    7552700.0   
                          ACGL     26.553333   26.680000     662100.0   
...                                      ...         ...          ...   
2024-09-13 00:00:00+00:00 XYL     130.639999  131.690002     995300.0   
                          YUM     132.929993  133.479996    1811500.0   
                          ZBH     105.199997  105.199997    1460500.0   
                          ZBRA    337.190002  339.119995     267700.0   
                          ZTS     190.139999  190.570007    1725200.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2016-09-19 00:00:00+00:00 A              -0.001377        NaN       NaN   
                          AAPL           -0.003150        NaN       NaN   
                          ABBV           -0.048853        NaN       NaN   
                          ABT            -0.009109        NaN       NaN   
                          ACGL            0.000088        NaN       NaN   
...                                            ...        ...       ...   
2024-09-13 00:00:00+00:00 XYL             0.000064  46.719230  4.847775   
                          YUM             0.000029  49.267093  4.893265   
                          ZBH             0.000292  41.878811  4.637091   
                          ZBRA            0.000163  53.096168  5.781744   
                          ZTS             0.000081  61.987360  5.190030   

Price                               bb_mid   bb_high       atr      macd  \
date                      ticker                                           
2016-09-19 00:00:00+00:00 A            NaN       NaN       NaN       NaN   
                          AAPL         NaN       NaN       NaN       NaN   
                          ABBV         NaN       NaN       NaN       NaN   
                          ABT          NaN       NaN       NaN       NaN   
                          ACGL         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2024-09-13 00:00:00+00:00 XYL     4.894730  4.941686  0.983831 -0.782992   
                          YUM     4.910877  4.928488  0.202263 -0.250077   
                          ZBH     4.715390  4.793689 -0.517943 -0.588159   
                          ZBRA    5.829234  5.876725  0.131598 -0.041165   
                          ZTS     5.228861  5.267693  0.472786  0.841610   

Price                             dollar_volume  
date                      ticker                 
2016-09-19 00:00:00+00:00 A           81.382600 

## 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.

* To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [4]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2016-10-31 00:00:00+00:00 A           75.765037   41.018742         -0.001089   
                          AAPL      3494.595092   26.182171         -0.002541   
                          ABBV       274.543295   39.878784         -0.049190   
                          ABT        317.927996   34.112480         -0.008074   
                          ACGL        29.912385   25.990000          0.000021   
...                                         ...         ...               ...   
2024-09-30 00:00:00+00:00 XYL        167.600566  130.830002          0.000064   
                          YUM        256.852089  133.649994          0.000029   
                          ZBH        209.817211  106.260002          0.000292   
                          ZBRA       119.451139  337.480011          0.000163   
                          ZTS        362.086688  191.089996          0.000081   

                                        rsi    bb_low    bb_mid   bb_high  \
date                      ticker                                            
2016-10-31 00:00:00+00:00 A       36.757818  3.718486  3.778792  3.839099   
                          AAPL    43.065638  3.293123  3.322002  3.350880   
                          ABBV    25.702504  3.744517  3.798670  3.852823   
                          ABT     36.555435  3.549492  3.599959  3.650426   
                          ACGL    45.040417  3.278161  3.300339  3.322517   
...                                     ...       ...       ...       ...   
2024-09-30 00:00:00+00:00 XYL     46.719230  4.847775  4.894730  4.941686   
                          YUM     49.267093  4.893265  4.910877  4.928488   
                          ZBH     41.878811  4.637091  4.715390  4.793689   
                          ZBRA    53.096168  5.781744  5.829234  5.876725   
                          ZTS     61.987360  5.190030  5.228861  5.267693   

                                       atr      macd  
date                      ticker                      
2016-10-31 00:00:00+00:00 A      -1.369369 -0.605120  
                          AAPL   -1.294865 -0.242981  
                          ABBV   -1.030408 -0.782741  
                          ABT    -1.261149 -0.700135  
                          ACGL   -1.093069 -0.555769  
...                                    ...       ...  
2024-09-30 00:00:00+00:00 XYL     0.983831 -0.782992  
                          YUM     0.202263 -0.250077  
                          ZBH    -0.517943 -0.588159  
                          ZBRA    0.131598 -0.041165  
                          ZTS     0.472786  0.841610  

[47253 rows x 9 columns]

In [5]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi  \
date                      ticker                                            
2017-09-30 00:00:00+00:00 AAPL     36.166763         -0.001175  45.768774   
                          ABBV     65.390602         -0.035700  70.568511   
                          ABT      47.232559         -0.006208  63.665021   
                          ACN     121.064766         -0.005643  56.250212   
                          ADBE    149.179993          0.000055  47.932483   
...                                      ...               ...        ...   
2024-09-30 00:00:00+00:00 VZ       44.430000          0.000137  67.017513   
                          WFC      52.779999          0.000184  40.833267   
                          WMT      80.599998          0.000041  79.993442   
                          XOM     111.150002          0.000092  43.232470   
                          ZTS     191.089996          0.000081  61.987360   

                                    bb_low    bb_mid   bb_high       atr  \
date                      ticker                                           
2017-09-30 00:00:00+00:00 AAPL    3.589136  3.636917  3.684699 -1.155463   
                          ABBV    4.050517  4.152375  4.254232 -0.471472   
                          ABT     3.825775  3.851606  3.877438 -1.186482   
                          ACN     4.770444  4.805951  4.841457 -1.125643   
                          ADBE    4.977719  5.031808  5.085897 -1.399836   
...                                    ...       ...       ...       ...   
2024-09-30 00:00:00+00:00 VZ      3.705046  3.758661  3.812276 -0.039735   
                          WFC     3.974080  4.038918  4.103755  1.764195   
                          WMT     4.306262  4.352192  4.398122  1.521680   
                          XOM     4.704856  4.752671  4.800486  1.014722   
                          ZTS     5.190030  5.228861  5.267693  0.472786   

                                      macd  
date                      ticker            
2017-09-30 00:00:00+00:00 AAPL   -0.459764  
                          ABBV    1.022601  
                          ABT     0.329676  
                          ACN     0.162857  
                          ADBE   -0.297820  
...                                    ...  
2024-09-30 00:00:00+00:00 VZ      1.702497  
                          WFC    -0.937198  
                          WMT     3.788688  
                          XOM    -1.168109  
                          ZTS     0.841610  

[12665 rows x 8 columns]

In [7]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2016-10-31 00:00:00+00:00 A           75.765037   41.018742         -0.001089   
                          AAPL      3494.595092   26.182171         -0.002541   
                          ABBV       274.543295   39.878784         -0.049190   
                          ABT        317.927996   34.112480         -0.008074   
                          ACGL        29.912385   25.990000          0.000021   
...                                         ...         ...               ...   
2024-09-30 00:00:00+00:00 XYL        167.600566  130.830002          0.000064   
                          YUM        256.852089  133.649994          0.000029   
                          ZBH        209.817211  106.260002          0.000292   
                          ZBRA       119.451139  337.480011          0.000163   
                          ZTS        362.086688  191.089996          0.000081   

                                        rsi    bb_low    bb_mid   bb_high  \
date                      ticker                                            
2016-10-31 00:00:00+00:00 A       36.757818  3.718486  3.778792  3.839099   
                          AAPL    43.065638  3.293123  3.322002  3.350880   
                          ABBV    25.702504  3.744517  3.798670  3.852823   
                          ABT     36.555435  3.549492  3.599959  3.650426   
                          ACGL    45.040417  3.278161  3.300339  3.322517   
...                                     ...       ...       ...       ...   
2024-09-30 00:00:00+00:00 XYL     46.719230  4.847775  4.894730  4.941686   
                          YUM     49.267093  4.893265  4.910877  4.928488   
                          ZBH     41.878811  4.637091  4.715390  4.793689   
                          ZBRA    53.096168  5.781744  5.829234  5.876725   
                          ZTS     61.987360  5.190030  5.228861  5.267693   

                                       atr      macd  
date                      ticker                      
2016-10-31 00:00:00+00:00 A      -1.369369 -0.605120  
                          AAPL   -1.294865 -0.242981  
                          ABBV   -1.030408 -0.782741  
                          ABT    -1.261149 -0.700135  
                          ACGL   -1.093069 -0.555769  
...                                    ...       ...  
2024-09-30 00:00:00+00:00 XYL     0.983831 -0.782992  
                          YUM     0.202263 -0.250077  
                          ZBH    -0.517943 -0.588159  
                          ZBRA    0.131598 -0.041165  
                          ZTS     0.472786  0.841610  

[47253 rows x 9 columns]

* Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [8]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

adj close  garman_klass_vol        rsi  \
date                      ticker                                            
2017-09-30 00:00:00+00:00 AAPL     36.166763         -0.001175  45.768774   
                          ABBV     65.390602         -0.035700  70.568511   
                          ABT      47.232559         -0.006208  63.665021   
                          ACN     121.064766         -0.005643  56.250212   
                          ADBE    149.179993          0.000055  47.932483   
...                                      ...               ...        ...   
2024-09-30 00:00:00+00:00 VZ       44.430000          0.000137  67.017513   
                          WFC      52.779999          0.000184  40.833267   
                          WMT      80.599998          0.000041  79.993442   
                          XOM     111.150002          0.000092  43.232470   
                          ZTS     191.089996          0.000081  61.987360   

                                    bb_low    bb_mid   bb_high       atr  \
date                      ticker                                           
2017-09-30 00:00:00+00:00 AAPL    3.589136  3.636917  3.684699 -1.155463   
                          ABBV    4.050517  4.152375  4.254232 -0.471472   
                          ABT     3.825775  3.851606  3.877438 -1.186482   
                          ACN     4.770444  4.805951  4.841457 -1.125643   
                          ADBE    4.977719  5.031808  5.085897 -1.399836   
...                                    ...       ...       ...       ...   
2024-09-30 00:00:00+00:00 VZ      3.705046  3.758661  3.812276 -0.039735   
                          WFC     3.974080  4.038918  4.103755  1.764195   
                          WMT     4.306262  4.352192  4.398122  1.521680   
                          XOM     4.704856  4.752671  4.800486  1.014722   
                          ZTS     5.190030  5.228861  5.267693  0.472786   

                                      macd  
date                      ticker            
2017-09-30 00:00:00+00:00 AAPL   -0.459764  
                          ABBV    1.022601  
                          ABT     0.329676  
                          ACN     0.162857  
                          ADBE   -0.297820  
...                                    ...  
2024-09-30 00:00:00+00:00 VZ      1.702497  
                          WFC    -0.937198  
                          WMT     3.788688  
                          XOM    -1.168109  
                          ZTS     0.841610  

[12665 rows x 8 columns]